#### **Import Libraries**

In [ ]:
import os 
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from tqdm import tqdm
import numpy as np 
import random 
import math
from itertools import chain
from IPython.display import display, Markdown
import textwrap
import tiktoken
import csv
import time 
import pandas as pd 
import torch
from datasets import Dataset
from torch.utils.data import DataLoader
import transformers
from transformers import DataCollatorWithPadding
from llmft.train import EncoderTrainer, EarlyStopping
from llmft.metrics import compute_recall
from llmft.losses import FocalLoss
from llmft.utils import predict
import seaborn as sns 
import jax 
import jax.numpy as jnp 

#### **Set Up Paths**

In [ ]:
data_csv = './../../../toy-data/exp2/results/llms_combined_results.csv'

#### **Set Up Plotting**

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import rcParams
rcParams['image.interpolation'] = 'nearest'
rcParams['image.cmap'] = 'viridis'
rcParams['axes.grid'] = False
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('seaborn-v0_8-dark-palette')

from matplotlib import font_manager 
locations = './../../../styles/Newsreader'
font_files = font_manager.findSystemFonts(fontpaths=locations)
print(locations)
print(font_files[0])
for f in font_files: 
    font_manager.fontManager.addfont(f)
plt.rcParams["font.family"] = "Newsreader"

#### Read in Data

In [ ]:
df = pd.read_csv(data_csv)

df = df[:18000]
print(df.shape)

# def fstage(race, gender, rent, health, fault):
#     return 0.05 + 0.1*race + 0.2*race*fault + 0.1*race*gender + 0.1*health *fault * fault + 0.35*fault 
# df['FStage_Value'] = df.apply(lambda row: fstage(row['Race'], row['Gender'], row['Rent'], row['Health'], row['Fault']), axis=1)
# fig = plt.figure(dpi=300, tight_layout=True, figsize=(7, 4.5))
# ax = plt.axes(facecolor=(.95, .96, .97))

# # Plot customizations
# for key in 'left', 'right', 'top':
#     ax.spines[key].set_visible(False)
# ax.text(0., 1.02, s='Count', transform=ax.transAxes, size=14)
# ax.yaxis.set_tick_params(length=0)
# ax.yaxis.grid(True, color='white', linewidth=2)
# ax.set_axisbelow(True)
# plt.hist(df['FStage_Value'], color='#36454F')
# plt.xlim(0, 1)
# plt.xlabel('Probability of Treatment Given Instrument', size=14)
# plt.show()

In [ ]:
llm_residuals = jnp.array(df.Residuals.values).reshape(-1, 500, 1)
llm_ones = jnp.ones_like(llm_residuals)
llm_outcomes = jnp.array(df.Outcome.values).reshape(-1, 500, 1)
llm_regressors = jnp.hstack((llm_residuals, llm_ones))
print(llm_residuals.shape)
print(llm_ones.shape)
print(llm_regressors.shape)
print(llm_outcomes.shape)

In [ ]:
def lstsq(Ones, X, Y):
    regressors = jnp.hstack((X, Ones))
    return jnp.linalg.lstsq(regressors, Y, rcond=None)[0][0]

In [ ]:
llm_results = jax.vmap(lstsq, in_axes=(0, 0, 0))(llm_ones, llm_residuals, llm_outcomes)

In [ ]:
print(jnp.mean(llm_results)); print(jnp.std(llm_results))